<a href="https://colab.research.google.com/github/viswakimi/PowerPulse/blob/main/PowerPulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://www.kaggle.com/code/frozenblaze/analysis-of-various-ml-algo-on-household-power?scriptVersionId=212172692&cellId=5
https://www.kaggle.com/code/aadhilimam/time-series-analysis-visualization-forecasting


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Step 1: Load Data
data_path = "household_power_consumption.txt"  # Update this path to the dataset location

data = pd.read_csv(data_path, sep=';', low_memory=False)

# Step 2: Data Understanding
print(data.info())
print(data.head())

# Step 3: Handle Missing Values and Data Types
# Replace '?' with NaN and convert numeric columns
for col in ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2']:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Drop rows with missing values
data.dropna(inplace=True)

# Combine Date and Time into a datetime column
data['Datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%d/%m/%Y %H:%M:%S')
data.set_index('Datetime', inplace=True)

# Drop original Date and Time columns
data.drop(['Date', 'Time'], axis=1, inplace=True)

# Step 4: Feature Engineering
# Extract time-based features
data['Year'] = data.index.year
data['Month'] = data.index.month
data['Day'] = data.index.day
data['Hour'] = data.index.hour
data['Weekday'] = data.index.weekday

# Step 5: Visualization (EDA)
plt.figure(figsize=(10, 6))
sns.lineplot(x=data.index, y='Global_active_power', data=data, label='Global Active Power')
plt.title('Time-Series Plot of Global Active Power')
plt.xlabel('Datetime')
plt.ylabel('Global Active Power (kW)')
plt.legend()
plt.show()

# Step 6: Prepare Data for Modeling
X = data[['Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3',
          'Year', 'Month', 'Day', 'Hour', 'Weekday']]
y = data['Global_active_power']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 7: Model Training
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 8: Model Evaluation
predictions = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions))
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

# Step 9: Feature Importance
feature_importances = model.feature_importances_
features = X.columns

plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importances, y=features)
plt.title('Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

# Step 10: Save the Model
import joblib
joblib.dump(model, 'energy_consumption_model.pkl')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Load the dataset
data_path = "household_power_consumption.txt"  # Replace with your dataset path
data = pd.read_csv(data_path, sep=';', na_values='?', low_memory=False)

# Step 2: Data Preprocessing
# Parse Date and Time into a single Datetime column
data['Datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
data.drop(['Date', 'Time'], axis=1, inplace=True)

# Convert relevant columns to numeric
data.iloc[:, :-1] = data.iloc[:, :-1].apply(pd.to_numeric, errors='coerce')

# Drop rows with missing values
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

# Step 3: Feature Engineering
# Extract time-based features
data['Hour'] = data['Datetime'].dt.hour
data['Day'] = data['Datetime'].dt.day
data['Month'] = data['Datetime'].dt.month
data['Year'] = data['Datetime'].dt.year
data['Is_peak_hour'] = data['Hour'].apply(lambda x: 1 if 6 <= x <= 10 or 17 <= x <= 21 else 0)

# Set Datetime as index for resampling
data.set_index('Datetime', inplace=True)

# Create rolling averages and resampled features
data['Daily_avg_power'] = data['Global_active_power'].resample('D').transform('mean')
data['Weekly_avg_power'] = data['Global_active_power'].resample('W').transform('mean')
data['Rolling_avg_3h'] = data['Global_active_power'].rolling(window=3, min_periods=1).mean()
data['Rolling_avg_24h'] = data['Global_active_power'].rolling(window=24, min_periods=1).mean()

data.reset_index(inplace=True)

# Step 4: Define Features and Target
features = ['Hour', 'Day', 'Month', 'Year', 'Is_peak_hour', 'Sub_metering_1',
            'Sub_metering_2', 'Sub_metering_3', 'Daily_avg_power', 'Weekly_avg_power',
            'Rolling_avg_3h', 'Rolling_avg_24h']
target = 'Global_active_power'

X = data[features]
y = data[target]

# Step 5: Data Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 7: Model Training
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Step 8: Model Evaluation
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Performance:")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R-Squared: {r2:.4f}")

# Step 9: Feature Importance Analysis
feature_importances = pd.DataFrame({'Feature': features, 'Importance': model.feature_importances_})
feature_importances.sort_values(by='Importance', ascending=False, inplace=True)

# Plot feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances)
plt.title('Feature Importance')
plt.show()

# Step 10: Visualize Predictions
plt.figure(figsize=(10, 6))
plt.plot(y_test.values[:100], label='Actual', marker='o')
plt.plot(y_pred[:100], label='Predicted', marker='x')
plt.title('Actual vs Predicted Power Consumption')
plt.xlabel('Sample Index')
plt.ylabel('Global Active Power')
plt.legend()
plt.show()

# Save the model and scaler for future use
import joblib
joblib.dump(model, 'energy_forecast_model.pkl')
joblib.dump(scaler, 'scaler.pkl')


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming 'data' is the cleaned DataFrame with the necessary datetime features
data.set_index('Datetime', inplace=True)

# Feature Engineering
data['Daily_avg_power'] = data['Global_active_power'].resample('D').transform('mean')
data['Weekly_avg_power'] = data['Global_active_power'].resample('W').transform('mean')
data['Is_peak_hour'] = data['Hour'].apply(lambda x: 1 if (6 <= x <= 10 or 17 <= x <= 21) else 0)
data['Rolling_avg_3h'] = data['Global_active_power'].rolling(window=3, min_periods=1).mean()
data['Rolling_avg_24h'] = data['Global_active_power'].rolling(window=24, min_periods=1).mean()

data.reset_index(inplace=True)

# Selecting features and target
features = ['Hour', 'Day', 'Month', 'Year', 'Is_peak_hour',
            'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3',
            'Daily_avg_power', 'Weekly_avg_power', 'Rolling_avg_3h', 'Rolling_avg_24h']
target = 'Global_active_power'

X = data[features]
y = data[target]

# Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# Step 1: Load and Explore the Data
file_path = "household_power_consumption.txt"  # Replace with the path to your dataset
data = pd.read_csv(file_path, sep=';', low_memory=False)
data.columns = data.columns.str.strip()  # Strip whitespace from column names

# Split combined column into separate columns
data[['Date', 'Time', 'Global_active_power', 'Global_reactive_power', 'Voltage',
      'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']] = \
    data.iloc[:, 0].str.split(';', expand=True)
data.dropna(inplace=True)  # Drop rows with missing values

# Convert data types
data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%d/%m/%Y %H:%M:%S')
data['Global_active_power'] = pd.to_numeric(data['Global_active_power'], errors='coerce')
data['Voltage'] = pd.to_numeric(data['Voltage'], errors='coerce')
data['Global_intensity'] = pd.to_numeric(data['Global_intensity'], errors='coerce')
data['Sub_metering_1'] = pd.to_numeric(data['Sub_metering_1'], errors='coerce')
data['Sub_metering_2'] = pd.to_numeric(data['Sub_metering_2'], errors='coerce')
data['Sub_metering_3'] = pd.to_numeric(data['Sub_metering_3'], errors='coerce')
data.dropna(inplace=True)  # Handle invalid data conversions

# Step 2: Exploratory Data Analysis (EDA)
print(data.info())
print(data.describe())

# Visualize the correlations
correlation_matrix = data.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Feature Correlation Matrix")
plt.show()

# Step 3: Feature Engineering
data['Hour'] = data['DateTime'].dt.hour
data['DayOfWeek'] = data['DateTime'].dt.dayofweek
data['Month'] = data['DateTime'].dt.month

# Aggregated features
data['Daily_Avg_Power'] = data.groupby(data['DateTime'].dt.date)['Global_active_power'].transform('mean')

# Step 4: Data Preprocessing
features = ['Global_intensity', 'Voltage', 'Hour', 'DayOfWeek', 'Month',
            'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
target = 'Global_active_power'

X = data[features]
y = data[target]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 5: Model Training
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 6: Model Evaluation
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

# Visualize predictions vs. actual values
plt.figure(figsize=(10, 6))
plt.plot(y_test.values[:100], label='Actual', marker='o')
plt.plot(y_pred[:100], label='Predicted', marker='x')
plt.title('Actual vs Predicted Energy Consumption')
plt.xlabel('Sample Index')
plt.ylabel('Energy Consumption (kW)')
plt.legend()
plt.show()

# Feature Importance
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})
feature_importance_df.sort_values(by='Importance', ascending=False, inplace=True)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance_df, x='Importance', y='Feature', palette='viridis')
plt.title('Feature Importance')
plt.show()

# Insights
print("Key Insights:")
print("- High usage hours and days of the week can help consumers adjust their habits.")
print("- Voltage stability plays a role in energy consumption trends.")
print("- Sub-metering data can help identify specific areas of high energy use.")


In [35]:
file_path = "/content/drive/MyDrive/guvii/projects dataset/household_power_consumption.txt"  # Replace with the path to your dataset
data = pd.read_csv(file_path, sep=';', low_memory=False)
data.columns = data.columns.str.strip()  # Strip whitespace from column names

In [36]:
data.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    object 
 3   Global_reactive_power  object 
 4   Voltage                object 
 5   Global_intensity       object 
 6   Sub_metering_1         object 
 7   Sub_metering_2         object 
 8   Sub_metering_3         float64
dtypes: float64(1), object(8)
memory usage: 142.5+ MB


In [26]:
data.shape

(2075259, 9)

In [37]:
data.isnull().sum()

,0
Date,0
Time,0
Global_active_power,0
Global_reactive_power,0
Voltage,0
Global_intensity,0
Sub_metering_1,0
Sub_metering_2,0
Sub_metering_3,25979


In [38]:
# Convert numeric columns
numeric_columns = ['Global_active_power', 'Global_reactive_power', 'Voltage',
                   'Global_intensity', 'Sub_metering_1', 'Sub_metering_2']
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    float64
 3   Global_reactive_power  float64
 4   Voltage                float64
 5   Global_intensity       float64
 6   Sub_metering_1         float64
 7   Sub_metering_2         float64
 8   Sub_metering_3         float64
dtypes: float64(7), object(2)
memory usage: 142.5+ MB


In [41]:
data.isnull().sum()

,0
Date,0
Time,0
Global_active_power,25979
Global_reactive_power,25979
Voltage,25979
Global_intensity,25979
Sub_metering_1,25979
Sub_metering_2,25979
Sub_metering_3,25979


In [42]:
missing_data = data[data.isnull().any(axis=1)]
print(missing_data)


               Date      Time  Global_active_power  Global_reactive_power  \
6839     21/12/2006  11:23:00                  NaN                    NaN   
6840     21/12/2006  11:24:00                  NaN                    NaN   
19724    30/12/2006  10:08:00                  NaN                    NaN   
19725    30/12/2006  10:09:00                  NaN                    NaN   
41832     14/1/2007  18:36:00                  NaN                    NaN   
...             ...       ...                  ...                    ...   
1990185   28/9/2010  19:09:00                  NaN                    NaN   
1990186   28/9/2010  19:10:00                  NaN                    NaN   
1990187   28/9/2010  19:11:00                  NaN                    NaN   
1990188   28/9/2010  19:12:00                  NaN                    NaN   
2027411  24/10/2010  15:35:00                  NaN                    NaN   

         Voltage  Global_intensity  Sub_metering_1  Sub_metering_2  \
6839 

In [43]:
missing_data.head(20)

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
6839,21/12/2006,11:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6840,21/12/2006,11:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19724,30/12/2006,10:08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19725,30/12/2006,10:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41832,14/1/2007,18:36:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61909,28/1/2007,17:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98254,22/2/2007,22:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98255,22/2/2007,22:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142588,25/3/2007,17:52:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190497,28/4/2007,00:21:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
data.head(30)

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
5,16/12/2006,17:29:00,3.520,0.522,235.02,15.0,0.0,2.0,17.0
6,16/12/2006,17:30:00,3.702,0.520,235.09,15.8,0.0,1.0,17.0
7,16/12/2006,17:31:00,3.700,0.520,235.22,15.8,0.0,1.0,17.0
8,16/12/2006,17:32:00,3.668,0.510,233.99,15.8,0.0,1.0,17.0
9,16/12/2006,17:33:00,3.662,0.510,233.86,15.8,0.0,2.0,16.0


In [46]:
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M:%S').dt.time

In [48]:
data.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Hour
0,2006-12-16,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0,17
1,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0,17
2,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0,17
3,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0,17
4,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0,17


In [51]:
import pandas as pd

# Assuming 'data' is your original DataFrame with a 'Time' column in datetime.time format

# If Time is in datetime.time format, directly extract the hour
data['Hour'] = data['Time'].apply(lambda x: x.hour if isinstance(x, pd.Timestamp) else pd.to_datetime(str(x), format='%H:%M:%S').hour)

# Group by 'Hour' and fill missing values with the mean of each hour
data_cleaned = data.groupby('Hour').apply(lambda group: group.fillna(group.mean()))

# Reset the index to maintain the original structure
data_cleaned = data_cleaned.reset_index(drop=True)

# Display the cleaned data
print(data_cleaned.head())


KeyboardInterrupt: 

In [30]:
# Forward fill
data['Sub_metering_3'] = data['Sub_metering_3'].ffill()  # Apply forward fill

# Backward fill
data['Sub_metering_3'] = data['Sub_metering_3'].bfill()  # Apply backward fill

# Fill remaining missing values with the mean
data['Sub_metering_3'] = data['Sub_metering_3'].mean()  # Fill remaining NaN with mean


In [33]:
data.isnull().sum()

,0
Date,0
Time,0
Global_active_power,25979
Global_reactive_power,25979
Voltage,25979
Global_intensity,25979
Sub_metering_1,25979
Sub_metering_2,25979
Sub_metering_3,0


In [ ]:
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M:%S').dt.time

In [ ]:
# Add a combined datetime column
data['Datetime'] = pd.to_datetime(data['Date'].astype(str) + ' ' + data['Time'].astype(str))

In [ ]:
# Create new features: Hour, Day, Month, Year
data['Hour'] = data['Datetime'].dt.hour
data['Day'] = data['Datetime'].dt.day
data['Month'] = data['Datetime'].dt.month
data['Year'] = data['Datetime'].dt.year

In [ ]:
# Example feature: Is_peak_hour (e.g., 6-9 AM and 5-9 PM)
data['Is_peak_hour'] = data['Hour'].apply(lambda x: 1 if (6 <= x <= 9 or 17 <= x <= 21) else 0)


In [ ]:
# Step 6: Verify the final dataset
print(data.head())
print(data.info())

        Date      Time  Global_active_power  Global_reactive_power  Voltage  \
0 2006-12-16  17:24:00                4.216                  0.418   234.84   
1 2006-12-16  17:25:00                5.360                  0.436   233.63   
2 2006-12-16  17:26:00                5.374                  0.498   233.29   
3 2006-12-16  17:27:00                5.388                  0.502   233.74   
4 2006-12-16  17:28:00                3.666                  0.528   235.68   

   Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  \
0              18.4             0.0             1.0            17.0   
1              23.0             0.0             1.0            16.0   
2              23.0             0.0             2.0            17.0   
3              23.0             0.0             1.0            17.0   
4              15.8             0.0             1.0            17.0   

             Datetime  Hour  Day  Month  Year  Is_peak_hour  
0 2006-12-16 17:24:00    17   16    